**[문제 접근 방식]**
1. 통계 기반 모델 : LOF, ISF, GMM, K-Means, One Class SVM, Elliptic Envelope
---
2. AutoEncoder 기반 모델 : AE / VAE / DAGMM (Deep Autoencoder Gaussian Mixture Model)
3. Over-sampling & Classification : SMOTE +  sklearn  
4. 기타 앙상블  

=> 이후에 앙상블을 해서 성능을 높일 예정

참고  
1. [블로그](https://velog.io/@bgly/%EB%8D%B0%EC%9D%B4%EC%BD%98%EA%B2%BD%EC%97%B0%EB%8C%80%ED%9A%8C%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C-%EC%82%AC%EA%B8%B0-%ED%83%90%EC%A7%80-%ED%83%90%EC%83%89-m4m7r92x)  
2. [깃허브](https://github.com/jailies/Dacon_Credit_Card_Fraud_Detection/blob/main/%EB%B0%9C%ED%91%9C%20pdf/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%82%AC%EA%B8%B0%EA%B1%B0%EB%9E%98%ED%83%90%EC%A7%80AI_%EC%B5%9C%EC%A2%85%EB%B0%9C%ED%91%9C.pdf)  
3. 데이콘 코드 공유  

# 데이터

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/daicon/train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('/content/drive/MyDrive/daicon/val.csv')
val_df = val_df.drop(columns=['ID'])

In [4]:
test_df = pd.read_csv('/content/drive/MyDrive/daicon/test.csv')

In [5]:
val_x = val_df.drop(columns=['Class'])
val_y = val_df['Class']

# 통계 기반 모델

In [6]:
def get_pred_label(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [7]:
val_normal, val_fraud = val_df['Class'].value_counts() # 정상치, 이상치
val_contamination = val_fraud / val_normal

## LOF(Local Outlier Factor)

>배경  

<img src="https://velog.velcdn.com/images%2Fvvakki_%2Fpost%2F89a329f8-5f06-4686-a8a6-47cdd6d25561%2Fimage.png" width="300" height="300">  
- C1 : 밀도가 낮은 그룹, C2 : 밀도가 높은 그룹  
- O1, O2 : 이상치  
- 위 그림에서 (C2,O2) 거리는 C1 그룹 내 관측치 간의 거리와 유사하기 때문에, <u>전체 관점에서 O2는 이상치로 보기 어렵다.</u>  
- 이런 단점을 해결하고자 LOF를 이용하며, 이는 **국소적인 정보를 이용해** 이상치 정도를 나태낸다.  

> 정의  

- Local Outlier Factor(LOF)란 Outlier의 정도를 나타내되, 관측치의 주변 데이터를 이용해 국소적(local) 관점으로 이상치 정도 파악한다.  
- 주변 데이터를 몇 개까지 고려할 것인지 하이퍼 파라미터 $k$를 이용해 결정할 수 있다.  

<br>

---
출처 : https://velog.io/@vvakki_/LOFLocal-Outlier-Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
# 모델 학습
lof = LocalOutlierFactor(contamination = val_contamination)
lof_val_pred = lof.fit_predict(val_x)
lof_val_pred = get_pred_label(lof_val_pred)

In [ ]:
# 모델 결과 확인
unique, counts = np.unique(lof_val_pred, return_counts=True)
lof_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{lof_contamination}]')

클래스 : [0 1], 클래스별 개수 : [28431    31]
정상 데이터 : 28431, 사기 데이터 : 31
검증 데이터의 사기 비율 : [0.001090359115050473]


In [ ]:
# 모델 성능
print(classification_report(val_df['Class'], lof_val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.06      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462



## ISF(Isolation Forest)

> 정의   

Isolation Forest는 이름에서 볼 수 있듯이 tree 기반으로 구현되는데, 랜덤으로 데이터를 split하여 모든 관측치를 고립시키며 구현한다.

특히, 변수가 많은 데이터에서도 효율적으로 작동할 수 있는 장점이 있다.  
<br>
<img src="https://velog.velcdn.com/images%2Fvvakki_%2Fpost%2F053cffc6-0c51-448d-ac80-37c2bd9226bf%2Fimage.png" width="400" height="200">  
- **각 관측치를 분리시키는 것은 이상치가 정상치보다 쉽다**  
- (1) 정상 데이터 분리 : 약 7번의 split 필요  
- (2) 이상치 분리 : 약 3번의 split 필요  

<br>

> 동작 과정  

<img src="https://velog.velcdn.com/images%2Fvvakki_%2Fpost%2Fc59d0a7f-7a1c-4589-b799-cf40c6463d26%2Fimage.png" width="500" height="200">  
IF는 iTree를 여러 번 반복하여 앙상블 한다.  
(1) 서브샘플링 : 비복원 추출로 데이터 중 일부를 샘플링  
(2) 변수 선택 : 변수 q를 랜덤 선택  
(3) split point 설정 : 변수 q의 범위 중 uniform하게 split point 선택  
(4) (1)-(3) 과정을 모든 관측치가 split되거나, 임의의 split 횟수까지 반복하며 경로길이를 모두 저장 => iTree  
(5) (1)-(4) 과정(iTree)을 여러번 반복 => Isolation Forest

---
출처 : https://velog.io/@vvakki_/Isolation-Forest-%EB%AF%B8%EC%99%84%EC%84%B1

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
# 모델 학습
IF = IsolationForest(contamination=val_contamination, random_state=42)
IF.fit(train_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.0010551491277433877, random_state=42)

In [ ]:
IF_val_pred = IF.predict(val_x)
IF_val_pred = get_pred_label(IF_val_pred)

In [ ]:
# 모델 결과
unique, counts = np.unique(IF_val_pred, return_counts=True)
IF_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{IF_contamination}]')

클래스 : [0 1], 클래스별 개수 : [28435    27]
정상 데이터 : 28435, 사기 데이터 : 27
검증 데이터의 사기 비율 : [0.0009495340249692281]


In [ ]:
print(classification_report(val_y, IF_val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.37      0.33      0.35        30

    accuracy                           1.00     28462
   macro avg       0.68      0.67      0.68     28462
weighted avg       1.00      1.00      1.00     28462



## GMM(Gaussian Mixture Model)

>정의  

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99BC00385AC75F9A03" width="500" height="200">  
- 현실에 존재하는 복잡한 형태의 확률 분포를 K개의 Gaussian distribution을 혼합하여 표현(기본 개념)    
- 관측된 데이터가 특정 확률 분포를 이용해서 생성되었다고 가정하는 확률 모델  
- GMM에서 주요 파라미터인 평균, 분산, $\pi$(각 관측치가 각 분포에 속할 확률)를 사용하기 위해 MLE를 사용한다.  

<br>

> EM(Expectation Maximization) 방법  

- 혼합모형에서 파라미터를 찾기 위해 사용하는 방법으로, log-likelihood를 최대화할 때 아래 값을 먼저 계산해야 한다.  
$$p(z_i=k|x_i,\pi_{i,k},\mu_k,\sigma_k)$$
- | 뒤 부분 값들을 임의의 값으로 시작해서 zi를 업데이트하며 log likelihood가 최대일 때 파라미터를 찾는다.  
- 파라미터가 특정 값으로 수렴할 때까지 반복하면, 최종적으로 수렴하는 $\pi$를 찾을 수 있다. $\pi$를 알면, 특정 관측치가 어떤 분포에 속하는지 알 수 있어 군집 할당이 가능하다.  

<br>

> 군집 성능 평가  

- AIC(Akaike Information Criterion)나 BIC(Bayesian Information Criterion)  

<br>

---
출처 : https://yeong-jin-data-blog.tistory.com/entry/GMM-Gaussian-Mixture-Models

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# 모델 학습
gmm = GaussianMixture(n_components=2, random_state=42)
gmm.fit(train_df)
gmm_val_pred = gmm.predict(val_x)

In [ ]:
# 모델 결과
unique, counts = np.unique(gmm_val_pred, return_counts=True)
gmm_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{gmm_contamination}]')

클래스 : [0 1], 클래스별 개수 : [17872 10590]
정상 데이터 : 17872, 사기 데이터 : 10590
검증 데이터의 사기 비율 : [0.5925470008952551]


In [ ]:
# 모델 성능
print(classification_report(val_y, gmm_val_pred))

              precision    recall  f1-score   support

           0       1.00      0.63      0.77     28432
           1       0.00      0.90      0.01        30

    accuracy                           0.63     28462
   macro avg       0.50      0.76      0.39     28462
weighted avg       1.00      0.63      0.77     28462



## K-Means

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
# 모델 학습
kmeans = KMeans(n_clusters=2, init='k-means++',max_iter=300, random_state=42)
kmeans.fit(train_df)
kmeans_val_pred = kmeans.predict(val_x)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# 모델 결과
unique, counts = np.unique(kmeans_val_pred, return_counts=True)
kmeans_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{kmeans_contamination}]')

클래스 : [0 1], 클래스별 개수 : [27911   551]
정상 데이터 : 27911, 사기 데이터 : 551
검증 데이터의 사기 비율 : [0.019741320626276378]


In [ ]:
# 모델 성능
print(classification_report(val_y, kmeans_val_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     28432
           1       0.00      0.00      0.00        30

    accuracy                           0.98     28462
   macro avg       0.50      0.49      0.49     28462
weighted avg       1.00      0.98      0.99     28462



## One Class SVM(One Class Support Vector Machine)

>정의  

- SVM 개념을 이용해 라벨링되어 있지 않은 데이터를 클러스터링 하는 방법  
<img src="https://losskatsu.github.io/assets/images/ml/oneclass-svm/ocsvm12.png" width="300" height="300">  
- 위와 같이 라벨링이 되어 있지 않은 데이터들을 SVM을 활용하여 정상치와 이상치를 구분할 수 있다. 정상 데이터와 이상치 데이터를 나누는 초평면(hyperplane)을 생성해 분류할 수 있다.  


>개념  

<img src="https://losskatsu.github.io/assets/images/ml/oneclass-svm/ocsvm15.png" width="400" height="300">  

<br>

---
출처 : https://losskatsu.github.io/machine-learning/oneclass-svm/#3-one-class-svm%EC%9D%98-%EA%B0%9C%EB%85%90

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
# 모델 학습
ocsvm = OneClassSVM()
ocsvm.fit(train_df)
ocsvm_val_pred = ocsvm.predict(val_x)
ocsvm_val_pred = get_pred_label(ocsvm_val_pred)

In [ ]:
# 모델 결과
unique, counts = np.unique(ocsvm_val_pred, return_counts=True)
unique, counts
ocsvm_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ocsvm_contamination}]')

클래스 : [0 1], 클래스별 개수 : [14338 14124]
정상 데이터 : 14338, 사기 데이터 : 14124
검증 데이터의 사기 비율 : [0.9850746268656716]


In [ ]:
# 모델 성능
print(classification_report(val_y, ocsvm_val_pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67     28432
           1       0.00      1.00      0.00        30

    accuracy                           0.50     28462
   macro avg       0.50      0.75      0.34     28462
weighted avg       1.00      0.50      0.67     28462



## Elliptic Envelope

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
# 모델 학습
ell = EllipticEnvelope(contamination=val_contamination)
ell.fit(train_df)
ell_val_pred = ell.predict(val_x)
ell_val_pred = get_pred_label(ell_val_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-214.722163891988686 > -246.002372548154568). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-253.665171100307020 > -274.457854046523039). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:184: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-247.059645390985963 > -277.247999067034073). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/usr/local

In [ ]:
# 모델 결과
unique, counts = np.unique(ell_val_pred, return_counts=True)
ell_contamination = counts[1] / counts[0]

In [ ]:
print(f'클래스 : {unique}, 클래스별 개수 : {counts}')
print(f'정상 데이터 : {counts[0]}, 사기 데이터 : {counts[1]}')
print(f'검증 데이터의 사기 비율 : [{ell_contamination}]')

클래스 : [0 1], 클래스별 개수 : [28438    24]
정상 데이터 : 28438, 사기 데이터 : 24
검증 데이터의 사기 비율 : [0.0008439412054293551]


In [ ]:
# 모델 성능
print(classification_report(val_y, ell_val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.00      0.00      0.00        30

    accuracy                           1.00     28462
   macro avg       0.50      0.50      0.50     28462
weighted avg       1.00      1.00      1.00     28462



## 성능 결과

f1_macro  
- LOF : 0.53  
- ISF : **0.68**  
- GMM : 0.39  
- K-Means : 0.49  
- One Class SVM : 0.34  
- Elliptic Envelope : 0.50  

# Auto Encoder

In [1]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [8]:
## gpu 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [9]:
EPOCHS = 400
LR = 1e-2
BS = 16384 # 2^(x)
SEED = 41

In [10]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(SEED) # seed 고정

In [11]:
train_df.values

array([[ -1.35835406,  -1.34016307,   1.77320934, ...,  -0.05975184,
          4.98372109,  -0.99497175],
       [ -0.96627171,  -0.18522601,   1.79299334, ...,   0.06145763,
          1.41829106,  -0.99497175],
       [ -0.42596588,   0.96052304,   1.14110934, ...,   0.08108026,
         -0.25613079,  -0.99496   ],
       ...,
       [ -0.24192292,   0.71224653,   0.39980566, ...,   0.08126506,
         -0.23069936,   1.03490408],
       [  0.12031638,   0.93100513,  -0.54601214, ...,   0.06880333,
         -0.26982463,   1.03493932],
       [-11.88111789,  10.07178497,  -9.83478346, ...,   0.82373096,
         -0.29665339,   1.03495107]])

In [12]:
class MyDataset(Dataset):
  def __init__(self,df,eval_mode): # 클래스 생성할 때 실행되는 생성자
    self.df = df
    self.eval_mode = eval_mode

    if self.eval_mode: ## val data의 경우에만 'Class' drop
      self.labels = self.df['Class'].values # label
      self.df = self.df.drop(columns=['Class']).values
    else:
      self.df = self.df.values

  def __getitem__(self,index): # 클래스 인덱스에 접근할 때 자동 호출
    if self.eval_mode: ## val data
       x = torch.from_numpy(self.df[index]).type(torch.FloatTensor)
       y = torch.FloatTensor([self.labels[index]])
       return x, y
      #self.x = self.df[index]
      #self.y = self.labels[index] # label
      #return torch.Tensor(self.x), self.y

    else:
      self.x = self.df[index]
      return torch.Tensor(self.x)

  def __len__(self):
    return len(self.df)

In [13]:
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=6) # 1회당 BS개의 데이터씩 분할, 순서는 섞어서 분할(shuffle)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [14]:
val_dataset = MyDataset(df=val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=6)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [32]:
class AutoEncoder(nn.Module): # nn.Module : Neural Network의 모든 것을 포괄하는 모든 신경망 모델의 Base Class
  def __init__(self):
    super(AutoEncoder, self).__init__() # 부모 클래스의 __init__() 매직 메소드를 자식 클래스의 __init__() 매직 메소드에서 실행 한다
    self.Encoder = nn.Sequential(
        nn.Linear(30,64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
    )
    self.Decoder = nn.Sequential( # encoder 복원
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        nn.Linear(64,30),
    )

  def forward(self,x):
    x = self.Encoder(x)
    x = self.Decoder(x)
    return x

In [33]:
class Trainer():
  def __init__(self,model,optimizer,train_loader,val_loader,scheduler,device):
    self.model = model
    self.optimizer = optimizer
    self.train_loader = train_loader
    self.val_loader = val_loader
    self.scheduler = scheduler
    self.device = device
    # loss function
    self.criterion = nn.L1Loss().to(self.device)

  def fit(self,):
    self.model.to(self.device)
    best_score = 0
    avg = 1 ##

    for epoch in range(EPOCHS):
      self.model.train()
      train_loss = []
      for x in iter(self.train_loader):
        x = x.float().to(self.device)
        self.optimizer.zero_grad()

        _x = self.model(x)
        loss = self.criterion(x, _x)

        loss.backward()
        self.optimizer.step()

        train_loss.append(loss.item())

      score = self.validation(self.model, 0.95)
      print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}]')

      if self.scheduler is not None:
        self.scheduler.step(score)

      #if best_score < score:
      #  best_score = score
      #  torch.save(model.module.state_dict(), './best_model.pth',_use_new_zipfile_serialization=False)

      if best_score <= score and avg > np.mean(train_loss):
        best_score = score
        avg = np.mean(train_loss)
        torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
        print('---------------------------')
        print(f'Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

  def validation(self, eval_model, thr):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    eval_model.eval()
    pred = []
    true = []

    with torch.no_grad():
      for x, y in iter(self.val_loader):
        x = x.float().to(self.device)

        _x = self.model(x)
        diff = cos(x,_x).cpu().tolist()
        batch_pred = np.where(np.array(diff)<thr,1,0).tolist()
        pred += batch_pred
        true += y.tolist()

    return f1_score(true, pred, average='macro')

In [ ]:
model = nn.DataParallel(AutoEncoder())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

In [ ]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.5370454745633262] Val Score : [0.0025295113983675546]
---------------------------
Train loss : [0.5370454745633262] Val Score : [0.0025295113983675546])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [1] Train loss : [0.35320542965616497] Val Score : [0.1132001484270115]
---------------------------
Train loss : [0.35320542965616497] Val Score : [0.1132001484270115])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [2] Train loss : [0.2670635006257466] Val Score : [0.3059038384339846]
---------------------------
Train loss : [0.2670635006257466] Val Score : [0.3059038384339846])


...

Epoch : [397] Train loss : [0.020200414316994802] Val Score : [0.9097393418694286]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [398] Train loss : [0.021272783566798483] Val Score : [0.9097393418694286]


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch : [399] Train loss : [0.020205540316445485] Val Score : [0.9097393418694286]


In [34]:
model = AutoEncoder()
model.load_state_dict(torch.load('/content/drive/MyDrive/daicon/best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder(
    (Encoder): Sequential(
      (0): Linear(in_features=30, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
    (Decoder): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=30, bias=True)
    )
  )
)

In [35]:
test_df = pd.read_csv('/content/drive/MyDrive/daicon/test.csv')
test_df= test_df.drop(columns=['ID'])

In [36]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

In [37]:
def prediction(model, thr, test_loader, device):
  model.to(device)
  model.eval()

  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  pred = []

  with torch.no_grad():
    for x in iter(test_loader):
      x = x.float().to(device)
      _x = model(x)
      diff = cos(x, _x).cpu().tolist()
      batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
      pred += batch_pred

  return pred

In [38]:
preds = prediction(model, 0.95, test_loader, device)

In [39]:
submit = pd.read_csv('/content/drive/MyDrive/daicon/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [46]:
val_df['Class']

0        0
1        0
2        0
3        0
4        0
        ..
28457    0
28458    0
28459    0
28460    0
28461    0
Name: Class, Length: 28462, dtype: int64

In [51]:
len(submit)

142503

In [50]:
len(preds)

142503

In [53]:
from sklearn.metrics import f1_score

f1_score(submit['Class'],preds,average='macro')

0.0024221380618695263

# Auto Encoder 변형

In [15]:
#### NEW
# 계층 정규화
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon) # 계층정규화 완료
        return self.weight * x + self.bias # wx+b

# 활성화 함수
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class AutoEncoder1(nn.Module):
    def __init__(self):
        super(AutoEncoder1, self).__init__()


        self.ln = LayerNorm(5000)
        self.ln1 = LayerNorm(3500)
        self.ln2 = LayerNorm(2000)
        self.ln3 = LayerNorm(1000)

        self.upblock1 = nn.Sequential(nn.Linear(30, 1000), nn.BatchNorm1d(1000),GELU())
        self.upblock2 = nn.Sequential(nn.Linear(1000,2000), nn.BatchNorm1d(2000),GELU())
        self.upblock3 = nn.Sequential(nn.Linear(2000,3500), nn.BatchNorm1d(3500),GELU())
        self.upblock4 = nn.Sequential(nn.Linear(3500,5000), nn.BatchNorm1d(5000),GELU())

        self.downblock1 = nn.Sequential(nn.Linear(5000, 3500),nn.BatchNorm1d(3500),GELU())
        self.downblock2 = nn.Sequential(nn.Linear(3500, 2000),nn.BatchNorm1d(2000),GELU())
        self.downblock3 = nn.Sequential(nn.Linear(2000, 1000),nn.BatchNorm1d(1000),GELU())
        self.downblock4 = nn.Sequential(nn.Linear(1000, 300),nn.BatchNorm1d(300),GELU())

        self.fclayer = nn.Sequential(nn.Linear(300,30))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        upblock1_out = self.upblock1(x)
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)

        downblock1_out = self.downblock1(self.ln(upblock4_out))
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out
        downblock4_out = self.downblock4(self.ln3(skipblock3))

        x = self.fclayer(downblock4_out)

        return x # 4

In [16]:
##### NEW
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)

    def fit(self):
        self.model.to(self.device)
        best_score = 0
        avg = 1
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score <= score and avg > np.mean(train_loss):
                best_score = score
                avg = np.mean(train_loss)
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
                print('---------------------------')
                print(f'Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<thr, 1, 0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [17]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
import math

In [19]:
### NEW
import warnings
warnings.filterwarnings(action='ignore')

model = nn.DataParallel(AutoEncoder1())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.5130279106753213] Val Score : [0.01770493016168002])
---------------------------
Train loss : [0.5130279106753213] Val Score : [0.01770493016168002])
Epoch : [1] Train loss : [0.24230618136269705] Val Score : [0.2819864584893001])
---------------------------
Train loss : [0.24230618136269705] Val Score : [0.2819864584893001])
Epoch : [2] Train loss : [0.1678811631032399] Val Score : [0.49361894961452524])
---------------------------
Train loss : [0.1678811631032399] Val Score : [0.49361894961452524])
Epoch : [3] Train loss : [0.12743764796427318] Val Score : [0.5078511110326694])
---------------------------
Train loss : [0.12743764796427318] Val Score : [0.5078511110326694])
Epoch : [4] Train loss : [0.10653192549943924] Val Score : [0.5133525205302055])
---------------------------
Train loss : [0.10653192549943924] Val Score : [0.5133525205302055])
Epoch : [5] Train loss : [0.09206007846764155] Val Score : [0.5237181594008616])
---------------------------
T

In [21]:
model = AutoEncoder1()
model.load_state_dict(torch.load('/content/drive/MyDrive/daicon/best_model1.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder1(
    (ln): LayerNorm()
    (ln1): LayerNorm()
    (ln2): LayerNorm()
    (ln3): LayerNorm()
    (upblock1): Sequential(
      (0): Linear(in_features=30, out_features=1000, bias=True)
      (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock2): Sequential(
      (0): Linear(in_features=1000, out_features=2000, bias=True)
      (1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock3): Sequential(
      (0): Linear(in_features=2000, out_features=3500, bias=True)
      (1): BatchNorm1d(3500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock4): Sequential(
      (0): Linear(in_features=3500, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (downblock1): Sequentia

In [23]:
test_df = pd.read_csv('/content/drive/MyDrive/daicon/test.csv')
test_df= test_df.drop(columns=['ID'])

In [24]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

In [25]:
def prediction(model, thr, test_loader, device):
  model.to(device)
  model.eval()

  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  pred = []

  with torch.no_grad():
    for x in iter(test_loader):
      x = x.float().to(device)
      _x = model(x)
      diff = cos(x, _x).cpu().tolist()
      batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
      pred += batch_pred

  return pred

In [26]:
preds = prediction(model, 0.95, test_loader, device)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
#submit['Class'] = preds
#submit.to_csv('./submit_autoencoder.csv', index=False)